In [6]:
#Function for accessing csv files and reading them 
import pandas as pd
import numpy as np

def csvread(file):
    print(file)
    y = pd.read_csv(file)
    return y #y is a datframe


In [126]:
df = csvread('Data_isha\IABERDEE25_final.csv') 
#print(df)
numeric_features = df.select_dtypes(include=[np.number])


corr =numeric_features.corr()
print(corr['tempm'].sort_values(ascending=False)[:10], '\n')
print(corr['tempm'].sort_values(ascending=False)[-5:])


Data_isha\IABERDEE25_final.csv
tempm       1.000000
tempi       0.999955
dewpti      0.898105
dewpti.1    0.898105
dewptm      0.898049
wgustm      0.273034
wgusti      0.272950
Day         0.218386
wspdi       0.192421
wspdm       0.192134
Name: tempm, dtype: float64 

Year             -0.177809
heatindexm             NaN
heatindexi             NaN
solarradiation         NaN
uv                     NaN
Name: tempm, dtype: float64


In [127]:
#A trial way of extracting relevant features
import numpy as np
import statsmodels.api as sm

y = df['tempm']
#features chosen such that those who have high correlation with the target feature

x = [
     df['hum'],
     df['dewptm'],
     df['dewpti'],
     df['dewpti.1'],
     ]
def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(y, X).fit()
    return results


print(reg_m(y, x).summary())


C:\Users\Neeraj\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


                            OLS Regression Results                            
Dep. Variable:                  tempm   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 3.756e+06
Date:                Tue, 17 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:35:54   Log-Likelihood:                 13340.
No. Observations:               30984   AIC:                        -2.667e+04
Df Residuals:                   30980   BIC:                        -2.664e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.2620      0.009     30.358      0.0

In [128]:
import statsmodels.formula.api as sm
import collections
 
sliding_window_values_sens = collections.deque(maxlen=576)
#data = list(slidingwindow)
#for i in range(0,len(df)):
window_data=df
#window_data = sliding_window_values_sens.append(df)
#window_data.columns= [self.x1,self.x2,self.y]
window_data.columns= df.columns.values #just commentted to apply deque function
##self.concentrator.setLastSlidingWindow(window_data,self.sensorid)

#model fitting
####result = sm.ols(formula=self.query, data=window_data).fit()
y=df['tempm'] #giving all rows 
x=df[['tempi','dewptm','dewpti','dewpti.1','hum','wspdm','wspdi','wgustm','wgusti','wdird','pressurem','pressurei','windchillm', 'windchilli','heatindexm','heatindexi','precip_ratei','precip_ratem','precip_totalm','precip_totali']]
      
y.ix[576:]
x.ix[576:]
print(window_data.shape)
print(y.shape)
print(x.shape)
#result = sm.ols(formula='tempm ~ dewpti + hum + dewptm ', data=window_data).fit()  #used some features w
result = sm.ols(formula='y~x', data=window_data).fit()
#model parameters
print(result.params)
param_sensor=list(result.params)
print(param_sensor)
#localpred_sens=param_sensor[0]+values[0]*param_sensor[1]+values[1]*param_sensor[2]
#actual_value=values[2]

(30984, 33)
(30984,)
(30984, 20)
Intercept   -8.540737e-08
x[0]         5.543929e-01
x[1]         3.846579e-02
x[2]        -1.010054e-02
x[3]        -1.010054e-02
x[4]        -3.814039e-04
x[5]         2.512089e-03
x[6]        -3.790046e-03
x[7]         5.110744e-03
x[8]        -8.303480e-03
x[9]        -3.912563e-06
x[10]        4.009055e-03
x[11]       -1.349255e-01
x[12]        8.934668e-05
x[13]       -8.737229e-05
x[14]        8.539882e-04
x[15]        8.539882e-04
x[16]        3.282487e-01
x[17]       -1.346269e-02
x[18]        7.476757e-03
x[19]       -1.881109e-01
dtype: float64
[-8.5407366308824873e-08, 0.55439292256574846, 0.038465794491753358, -0.010100536117375492, -0.010100536117381487, -0.00038140386843804608, 0.0025120891403735968, -0.0037900462756992326, 0.0051107443110612882, -0.0083034804539252605, -3.912562594989271e-06, 0.0040090546938329474, -0.13492552803509739, 8.9346681497515235e-05, -8.7372285525688653e-05, 0.00085398817845823189, 0.00085398818072643292, 0.3282

C:\Users\Neeraj\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
C:\Users\Neeraj\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [8]:
#Was having problems with sliding window code used tail to get last 576 elements
#to get last 576 measurements and applying feature selection on that.
import statsmodels.formula.api as sm

df1=df.tail(576)
print(df1.shape)
y=df1['tempm'] #giving all rows 
x=df1[['tempi','dewptm','dewpti','dewpti.1','hum','wspdm','wspdi','wgustm','wgusti','wdird','pressurem','pressurei','windchillm', 'windchilli','heatindexm','heatindexi','precip_ratei','precip_ratem','precip_totalm','precip_totali']]
#y.tail(576)
#x.tail(576)
print(y.shape)
print(x.shape)
result = sm.ols(formula='y~x', data=df1).fit()
print(result.params)


(576, 33)
(576,)
(576, 20)
Intercept   -9.056944e-08
x[0]         5.538397e-01
x[1]        -2.484837e-02
x[2]         7.985741e-03
x[3]         7.985741e-03
x[4]        -7.198359e-06
x[5]        -1.888348e-01
x[6]         3.012899e-01
x[7]        -6.479458e-02
x[8]         1.048194e-01
x[9]         3.746202e-06
x[10]        4.868086e-02
x[11]       -1.652628e+00
x[12]       -5.674762e-04
x[13]        5.610327e-04
x[14]        9.056039e-04
x[15]        9.056032e-04
x[16]        0.000000e+00
x[17]        0.000000e+00
x[18]       -8.546227e-02
x[19]       -2.848742e-03
dtype: float64


In [132]:
#dropping columns:
df.drop(['Month', 'Date', 'Year', 'time_h', 'time_min', 'wdire', 'solarradiation','uv','Station'], axis=1, inplace=True)


In [135]:
##Is this the way to use sliding window?
import statsmodels.formula.api as sm
import collections
 
sliding_window_values_sens = collections.deque(maxlen=576)
for rows in df.iterrows():
    sliding_window_values_sens.append(rows) #to add last 576 measurements
#for row in df.itertuples(index=True, name='Pandas'):
   ### sliding_window_values_sens.append(row) #to add last 576 measurements
#print(sliding_window_values_sens)

In [136]:
#probelms in sliding window code
import statsmodels.formula.api as sm
import collections
 
sliding_window_values_sens = collections.deque(maxlen=576)
for rows in df.iterrows():
    sliding_window_values_sens.append(rows) #to add last 576 measurements
#for row in df.itertuples(index=True, name='Pandas'):
   ### sliding_window_values_sens.append(row) #to add last 576 measurements
#list(sliding_window_values_sens)
data = list(sliding_window_values_sens) #is not returning any value


window_data = pd.DataFrame(np.array(sliding_window_values_sens))

print(window_data.shape)

#window_data.columns= [self.x1,self.x2,self.y]
#self.concentrator.setLastSlidingWindow(window_data,self.sensorid)

#print(window_data)  Here : getting just 2 columns the month number and index
y=window_data['tempm']
x=window_data[['dewptm','dewpti','hum','wspdm','wspdi','wgustm','wgusti','wdird','pressurem','pressurei','windchillm', 'windchilli','heatindexm','heatindexi','precip_ratei','precip_ratem','precip_totalm','precip_totali']]

print(y.shape) 
#model fitting
#formula = "y ~ tempi+ tempm + dewptm + dewpti + hum +wspdm +wspdi + wgustm + wgusti + wdird + pressurei"
result = sm.ols(formula='y~x', data=window_data).fit()
#results = sm.ols(df.a, sm.add_constant(df[list('bcd')])).fit()
#model parameters
param_sensor=list(result.params)
print(param_sensor)
#localpred_sens=param_sensor[0]+values[0]*param_sensor[1]+values[1]*param_sensor[2]
#actual_value=values[2]

(576, 2)


KeyError: 'tempm'

In [ ]:
# sleecting from first row till the end: df.loc['Dean':] instead od dean give index!
#https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c

In [11]:
%reload_ext autoreload
%autoreload 2

from stream_fast_weight import StreamFastWeight
from gen_feat_weight import GenFeatWeight

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:

#to get last 576 measurements and applying feature selection on that.
import statsmodels.formula.api as sm

df1=df.tail(576)
print(df1.shape)
y=df1['tempm'] #giving all rows 
x=df1[['tempi','dewptm','dewpti','dewpti.1','hum','wspdm','wspdi','wgustm','wgusti','wdird','pressurem','pressurei','windchillm', 'windchilli','heatindexm','heatindexi','precip_ratei','precip_ratem','precip_totalm','precip_totali']]
#y.tail(576)
#x.tail(576)
print(y.shape)
print(x.shape)
result = sm.ols(formula='y~x', data=df1).fit()
predictions = result.predict(x).mean()
print(result.params)
print("Predictions",predictions)
result.summary()

(576, 24)
(576,)
(576, 20)
Intercept   -9.056944e-08
x[0]         5.538397e-01
x[1]        -2.484837e-02
x[2]         7.985741e-03
x[3]         7.985741e-03
x[4]        -7.198359e-06
x[5]        -1.888348e-01
x[6]         3.012899e-01
x[7]        -6.479458e-02
x[8]         1.048194e-01
x[9]         3.746202e-06
x[10]        4.868086e-02
x[11]       -1.652628e+00
x[12]       -5.674762e-04
x[13]        5.610327e-04
x[14]        9.056039e-04
x[15]        9.056032e-04
x[16]        0.000000e+00
x[17]        0.000000e+00
x[18]       -8.546227e-02
x[19]       -2.848742e-03
dtype: float64
Predictions 2.899826388892343


C:\Users\Neeraj\Anaconda3\lib\site-packages\statsmodels\base\model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Neeraj\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\Neeraj\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\Neeraj\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.752e+05
Date:                Sat, 14 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:02:42   Log-Likelihood:                 1254.4
No. Observations:                 576   AIC:                            -2479.
Df Residuals:                     561   BIC:                            -2414.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -9.057e-08   4.67e-09    -19.413      0.000   -9.97e-08   -8.14e-08
x[0]           0.5538      0.009     64.894      0.000       0.537       0.571
x[1]          -0.0248      0.041     -0.605      0.546      -0.106       0.056
x[2]           0.0080      0.012      0.654      0.513      -0.016       0.032
x[3]           0.0080      0.012      0.654      0.513      -0.016       0.032
x[4]       -7.198e-06      0.003     -0.003      0.998      -0.005       0.005
x[5]          -0.1888      0.105     -1.805      0.072      -0.394       0.017
x[6]           0.3013      0.168      1.793      0.073      -0.029       0.631
x[7]          -0.0648      0.048     -1.353      0.177      -0.159       0.029
x[8]           0.1048      0.077      1.360      0.174      -0.047       0.256
x[9]        3.746e-06   1.94e-05      0.193      0.847   -3.43e-05    4.18e-05
x[10]          0.0487      0.041      1.199      0.231      -0.031       0.128
x[11]         -1.6526      1.372     -1.205      0.229      -4.347       1.042
x[12]         -0.0006      0.003     -0.173      0.862      -0.007       0.006
x[13]          0.0006      0.003      0.178      0.859      -0.006       0.007
x[14]          0.0009   4.66e-05     19.413      0.000       0.001       0.001
x[15]          0.0009   4.66e-05     19.413      0.000       0.001       0.001
x[16]               0          0        nan        nan           0           0
x[17]               0          0        nan        nan           0           0
x[18]         -0.0855      0.054     -1.575      0.116      -0.192       0.021
x[19]         -0.0028      0.002     -1.575      0.116      -0.006       0.001
==============================================================================
Omnibus:                      186.053   Durbin-Watson:                   1.792
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               28.422
Skew:                          -0.017   Prob(JB):                     6.73e-07
Kurtosis:                       1.912   Cond. No.                     9.54e+40
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.28e-71. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [134]:
#code to access all files in a folder given the location of folder.
import os
#rootdir = 'C:/Users/sid/Desktop/test' 
rootdir = 'Data_isha'

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print(os.path.join(subdir, file)) to print names of all the csv files
        csvread(os.path.join(subdir, file))

Data_isha\IABERDEE25_final.csv
Data_isha\ICORNWAL26_final.csv
Data_isha\IDUNFERM18_final.csv
Data_isha\IEDINBUR99_final.csv
Data_isha\IGLASGOW1_final.csv
Data_isha\IHIGHLAN53_final.csv
Data_isha\IISLEOFS4_final.csv
Data_isha\ILONDON542_final.csv


In [ ]:
#df2 is an array!
df2 = x.reset_index().values
df2.dtype

In [35]:
df = csvread('Data_isha\IABERDEE25_final.csv')

 #x mese sari values ko threshold k against compare kar k pass karo yahi query hoga   

Data_isha\IABERDEE25_final.csv
